## Titanic: Machine Learning from Disaster

So, this is my first published notebook. I'm going to try to get the most of the training dataset to better understand the data. The focus of this notebook is to go through somethings I learned while studying data visualization in Python. There's a high chance that these codes aren't as pythonic as they are supposed to be, but I'll try my best to write some clean code. 

**Obs:** Critiques and tips are very welcome.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
%matplotlib inline

In [ ]:
df = pd.read_csv("../input/titanic/train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

### Notes
* There are 891 records on the dataset.
* There is missing information on the following columns: **Age**, **Cabin** and **Embarked**.

In [ ]:
# This fuctions is used a few times to display bar values in barplots
def show_values_on_bars(axs, *args, **kwargs):
    """
    Function based on Sharon Soussan answer for:
    https://stackoverflow.com/questions/43214978/seaborn-barplot-displaying-values
    This function will help display information over some plots.
    """
    
    def _show_on_single_plot(ax):        
        
        if 'stacked' in kwargs:
            if kwargs['stacked'] == True:
                first_bar = sorted([x.get_height() for x in axs.patches[:] if x.get_x() < 0], reverse=True)
                second_bar = sorted([x.get_height() for x in axs.patches[:] if x.get_x() > 0], reverse=True)

        for p in ax.patches:
            
            _x = p.get_x() + p.get_width() / 2
            
            if 'stacked' in kwargs:
                # This will work only for the sex/pclass plot. I will try to improve this in the future.
                if kwargs['stacked'] == True:
                    n = ax.patches.index(p)
                    if p.get_x() < 0:
                        bar = first_bar
                    else:
                        bar = second_bar
                    pos = bar.index(p.get_height())
                    if pos % 3 == 0:
                        _y = (bar[pos] - bar[pos + 1])/2 + bar[pos + 1] - 14
                        value = bar[pos] - bar[pos + 1]
                    elif pos % 3 == 1:
                        _y = (bar[pos] - bar[pos + 1])/2 + bar[pos + 1] - 14
                        value = bar[pos] - bar[pos + 1]
                    elif pos % 3 == 2:
                        _y = bar[pos] / 2 - 14
                        value = bar[pos]
                        
            elif 'height' in kwargs:
                _y = kwargs['height']
                value = f'{p.get_height()}'
            else:
                _y = 10
                value = f'{p.get_height()}'
            ax.text(_x, _y, value, fontsize=14, ha="center") 

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(axs)
    else:
        _show_on_single_plot(axs)

In [ ]:
# Setting the style for my plots
sns.set_style(style='ticks')

## Visualization based on 'Sex'

Some plots related to **Sex**. Let's check if 'women and children' firsts was a thing.

In [ ]:
df['Sex'].value_counts()

In [ ]:
# Color palettes for sex plots:
sex_palette = sns.color_palette(["#4287f5", "#bd3c3c"])
pclass_palette = sns.color_palette(["#FFDF00", "#c0c0c0", "#cd7f32"])
survived_palette = sns.color_palette(["#4e5245", "#57916c"])

fig = plt.figure(figsize=(16,12), constrained_layout=True)
gs = gridspec.GridSpec(nrows=3, ncols=4, figure=fig)


# Male/Female Totals
ax1 = fig.add_subplot(gs[0:1, 0:2])
sns.countplot(x='Sex', data=df, palette=sex_palette, edgecolor=sns.color_palette(["#000"]), alpha=0.5)
show_values_on_bars(ax1)
plt.ylabel('Passengers')

# Sex per class
ax2 = fig.add_subplot(gs[0:1, 2:])
sns.countplot(x = 'Sex', data=df, color = "#FFDF00", edgecolor=sns.color_palette(["#000"]), alpha=0.2)
sns.countplot(x = 'Sex', data=df[(df['Pclass'] == 2) | (df['Pclass'] == 3)], color='#808080', edgecolor=sns.color_palette(["#000"]), alpha=0.2)
sns.countplot(x = 'Sex', data=df[df['Pclass'] == 3], color = "#cd7f32", edgecolor=sns.color_palette(["#000"]), alpha=0.2)
plt.ylabel('Passengers')
# sns.countplot(x='Sex', hue='Pclass', data=df, edgecolor=sns.color_palette(["#000"]), palette=pclass_palette, alpha=0.5)
plt.legend(title='Per class', labels=['1st', '2nd', '3rd'])
show_values_on_bars(ax2, stacked=True)
plt.ylabel('Passengers')

# Survivors by sex
ax3 = fig.add_subplot(gs[1:3, 0:4])
sns.countplot(x='Sex', hue='Survived', data=df, edgecolor=sns.color_palette(["#000"]), palette=survived_palette, alpha=0.5)
plt.legend(title='Survived?', labels=['No', 'Yes'])
show_values_on_bars(ax3)
plt.ylabel('Passengers')

fig.suptitle('Sex', fontsize=18)

## Visualization based on 'Parch' and 'SibSp'

To better approach these columns, I decided to add both, considering the following given information:

* **sibsp:** The dataset defines family relations in this way...

    Sibling = brother, sister, stepbrother, stepsister
    
    Spouse = husband, wife (mistresses and fiancés were ignored)
    

* **parch:** The dataset defines family relations in this way...

    Parent = mother, father

    Child = daughter, son, stepdaughter, stepson
    

So, for the sake of the data visualization: **'Family'** = **'Parch'** + **'SibSp'**

**PS:** On a future notebook, I'll check if this is the best approach.

In [ ]:
df['Family'] = df['Parch'] + df['SibSp']

In [ ]:
df['Family'].value_counts()

In [ ]:
fig = plt.figure(figsize=(14,6), constrained_layout=True)
gs = gridspec.GridSpec(nrows=1, ncols=2, figure=fig)

ax1 = fig.add_subplot(gs[0, 0])
sns.countplot(x='Family', data=df, palette='coolwarm', edgecolor=sns.color_palette(["#000"]), alpha=0.5)
show_values_on_bars(ax1, height=40)
plt.xlabel('Family Members')
plt.ylabel('Passengers')

ax2 = fig.add_subplot(gs[0, 1])
sns.countplot(x = 'Family', data=df, color = "#5ec4a1", edgecolor=sns.color_palette(["#000"]), alpha=0.5)
sns.countplot(x = 'Family', data=df[df['Survived'] == 0], color = "black", edgecolor=sns.color_palette(["#000"]), alpha=0.5)
plt.xlabel('Family Members')
plt.ylabel('Passengers')
plt.legend(title='Survived?', labels=['Yes', 'No'])

fig.suptitle('Passengers Family', fontsize=18)

## Visualization based on 'Age'

Before applying data visualization techniques on the **Age** column, I decided to fill the 177 missing data records to have a more completed dataset.

**PS:** On a future notebook, I'll check if this is the best approach.

In [ ]:
# Filling na with age mean so there's no missing data on the 'Age' column
df['Age'].fillna(value=df['Age'].mean())

In [ ]:
df['Age'].value_counts()

In [ ]:
# Color palettes for age plots:
sex_palette = sns.color_palette(["#4287f5", "#bd3c3c"])
pclass_palette = sns.color_palette(["#FFDF00", "#c0c0c0", "#cd7f32"])
survived_palette = sns.color_palette(["#000", "#57cf81"])

fig = plt.figure(figsize=(20,6), constrained_layout=True)
gs = gridspec.GridSpec(nrows=1, ncols=3, figure=fig)

ax1 = fig.add_subplot(gs[0, 0])
sns.distplot(df['Age'], color='#000')

ax2 = fig.add_subplot(gs[0, 1])
sns.boxplot(x='Pclass', y='Age', hue='Sex', data=df, palette=sex_palette)

ax3 = fig.add_subplot(gs[0, 2])
sns.swarmplot(x='Pclass', y='Age', hue='Survived', data=df, palette=survived_palette, size=8, dodge=True)
plt.legend(title='Survived?', labels=['No', 'Yes'])

fig.suptitle('Age', fontsize=18)

## Correlation

Before using the heatmap over correlation, I'll give binary/ordinal representations to **Sex** and **Embarked** columns to check if there's any correlation between them and the **Survived** data.

In [ ]:
df['SexB'] = df['Sex'].map({'male': 1,'female': 0})
df['EmbarkedNum'] = df['Embarked'].map({'S': 0,'C': 1, 'Q': 2})

In [ ]:
sns.heatmap(df.corr(), cmap='Pastel1')
plt.title('Correlation', fontsize=24)

## Conclusions (so far)

Here are somethings I concluded by analysing the plots:

* Sex and Class matter to decide rather people would live or die (if you were a first-class woman, you'd probably survive);
* Age and Class are related (e.g. people in first class trend to be older than people in third class);
* Class and Fare are related (as expected, people in 1st class paid more for their tickets);
* Parch and SibSp doesn't appear to make such a difference to tell who survived or not.

I'll try to apply some machine learning algorithms on this dataset so I can learn more about it and maybe find more patterns.

## Bonus: Geographic Plots
The following charts don't matter for the analysis as a whole, I was just trying to train my geographical plot skills and I used some of the dataset information (and Googled a little more) to create figures somehow interesting.

In [ ]:
pip install chart_studio

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
df['Embarked'].value_counts()

In [ ]:
data = dict(type = 'choropleth',
           colorscale = 'portland',
           locations = ['Great Britain', 'France', 'Ireland'],
           locationmode = 'country names',
           z = [df['Embarked'].value_counts()['S'], df['Embarked'].value_counts()['C'], df['Embarked'].value_counts()['Q']],
           colorbar = {'title': 'Number of people'})
layout = dict(title = 'Where people embarked on Titanic', geo = dict(scope = 'europe', showlakes = True, lakecolor='rgb(85, 173, 240)')
             )
choromap = go.Figure(data=[data], layout=layout)
iplot(choromap)

In [ ]:
# C = Cherbourg (France) - Cordinates = (49.648793, -1.617686)
# Q = Queenstown/Cohb (Ireland) - Coordinates = (51.85033600, -8.29416667) 
# S = Southampton (England) - Coordinates = (50.90969800, -1.40416667)
# Titanic sinking coordinates - Coordinates = (41.7325, -49.9469)

name = ['Southampton', 
        'Cherbourg', 
        'Queenstown', 
        'Sinking']

pos = [[50.90969800, -1.40416667, 'Southampton', 1],
      [49.648793, -1.617686, 'Cherbourg', 2],
      [51.85033600, -8.29416667, 'Queenstown', 3],
      [41.7325, -49.9469, 'Titanic\'s Sinking', 4]]

pos_df = pd.DataFrame(pos, index=name, columns=['lat', 'lon', 'name', 'animation'])


fig = px.line_mapbox(pos_df, lat="lat", lon="lon", zoom=3, height=500, hover_data=['name'])
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=2.8, mapbox_center_lat = 48, mapbox_center_lon = -27,
    margin={"r":0,"t":0,"l":0,"b":0}, title='Titanic\'s Path')